In [1]:
import numpy as np
from qiskit import *
from math import pi
from qiskit.visualization import *
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute, BasicAer
from qiskit.compiler import transpile
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.extensions import RXGate, XGate, CXGate
from qiskit.ignis.mitigation.measurement import (complete_meas_cal,CompleteMeasFitter)

import time
import qiskit.quantum_info as qi
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity
from qiskit.providers.ibmq.managed import IBMQJobManager

/tmp/ipykernel_36229/3486435410.py:10: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.mitigation.measurement import (complete_meas_cal,CompleteMeasFitter)


In [2]:
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [3]:
provider = IBMQ.get_provider(hub='ibm-q-research', group='independent-12', project='main')

In [4]:
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-research', group='independent-12', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q-research', group='independent-12', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q-research', group='independent-12', project='main')>,
 <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q-research', group='independent-12', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-research', group='independent-12', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-research', group='independent-12', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-research', group='independent-12', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-research', group='independent-12', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-research', group='independent-12', project='main')>,
 <IBMQSimulator('simulator_extended_stabilizer') 

In [5]:
def A_gate(theta, qubits, circ, phi=0, l = 0, f = True):
    a = qubits[0] #this will be the first qubit
    b = qubits[1] #this will be the second
    
    if f == True:
        circ.cx(b,a) #I add the first CNOT, for which qubit b is the control and qubit a is the target
    else: 
        circ.x(a)
    #now comes the R dagger gate which has two parameters: theta and phi
    #but in our case phi is always zero
    ry1 = r"Y_"+str(l)+"^-1"
    ry  = r"Y_"+str(l)
    rz = r"Z_"+str(l)
    rz1 = r"Z_"+str(l)+"^-1"
    circ.ry(-(theta+np.pi/2),b, label = ry1) #rotation around the y axis
    circ.rz(-(phi+np.pi),b) #rotation around the z axis 
    
    #now the second CNOT for which target: b, control:a
    circ.cx(a,b)
    
    #and now the R gate for which we use the same two parameters
    circ.rz((phi+np.pi),b)
    circ.ry((theta+np.pi/2),b, label = ry)
    
    #and the last CNOT
    circ.cx(b,a)
    
    #we dont need to return the circuit itself because it is now updated

In [6]:
def circ_maker(n, qubits, measure = False, f = True, classic = False): 
    
    theta1 = 3*np.pi/4
    theta2 = -2.97167
    theta3 = -2.18628 
    theta4 = np.pi
    theta5 = 5.23599
    
    a = qubits[0]
    b = qubits[1]
    c = qubits[2]
    d = qubits[3]
    
    qreg = QuantumRegister(n)
    if classic == True: 
        creg = ClassicalRegister(4)
        circ = QuantumCircuit(qreg, creg)
    else: 
        circ = QuantumCircuit(qreg)
    circ.x(b)
    circ.x(d)
    
    A_gate(theta=theta1, qubits = [a,b], circ = circ, l = 1, f = f)
    A_gate(theta=theta2, qubits = [c,d], circ = circ, l = 2, f = f)
    A_gate(theta=theta3, qubits = [b,c], circ = circ, l = 3)
    A_gate(theta=theta4, qubits = [a,b], circ = circ, l = 4)
    A_gate(theta=theta5, qubits = [c,d], circ = circ, l = 5)
    A_gate(theta=theta3, qubits = [b,c], circ = circ, l = 3)

    if measure == True:
        circ.measure([a,b,c,d], [0,1,2,3])
    else: 
        pass 
    if classic == True:
        return circ, qreg, creg
    else: 
        return circ, qreg

In [7]:
def saver(filename, backend_name, time, shots, qubits, F, rho_mes, target_state, tomo_fitter_data, F_mit, 
          rho_mit, cal_matrix):  
    with open (filename+".txt", "w") as f: 
        f.write("Backend: "+backend_name+"\n")
        f.write("date: "+ str(time)+"\n")
        f.write("Qubits in use: "+str(qubits)+"\n")
        f.write("Shots: "+str(shots)+"\n")
        f.write("\n")
        f.write("Target state: "+ str(target_state)+"\n"+"\n")
        f.write("Fidelity: "+str(F)+"\n")
        f.write("Mitigated Fidelity: "+str(F_mit)+"\n"+"\n")
        
        f.write("Measured density matrix:"+ "\n")
        for i, line in enumerate(rho_mes):
            f.write(str(line))
            f.write("\n")
        f.write("\n")
            
        f.write("Mitigated density matrix:"+"\n")
        for i, line in enumerate(rho_mit):
            f.write(str(line))
            f.write("\n")
                    
        f.write("\n")
        f.write("Measurement results for the circuits:"+ "\n")
        
        
        #print(dictionary, file = f)
        for key in tomo_fitter_data:
            f.write(str(key)+": "+"\n")
            for subkey in tomo_fitter_data[key]:
                f.write("\t"+str(subkey)+": "+str(tomo_fitter_data[key][subkey])+"\n")
                
        f.write("\n")
        
        f.write("Calibration matrix:"+"\n")
        for line in cal_matrix:
            f.write(str(line)+"\n")

In [8]:
circ, qreg = circ_maker(n = 4, qubits = [0,1,2,3])
job = execute(circ, backend = Aer.get_backend('statevector_simulator'))
target_state = job.result().get_statevector(circ)
print(target_state)

Statevector([-1.25196530e-17+2.93780988e-17j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
             -5.77350310e-01-2.90185755e-16j,
              0.00000000e+00+0.00000000e+00j,
              2.88675671e-01+1.24752636e-16j,
              2.88674690e-01+2.16227266e-16j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              2.88674517e-01+8.26955458e-17j,
              2.88668804e-01+2.60761812e-16j,
              0.00000000e+00+0.00000000e+00j,
             -5.77353657e-01-5.55801360e-16j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              2.12462951e-20-2.98888736e-18j],
            dims=(2, 2, 2, 2))


## Santiago 

In [20]:
backend = provider.get_backend('ibmq_santiago')
backend_name = 'Santiago'

time = "2022.06.01."
n=5
shots = 8192

filename = "Athens1"
qubits = np.array([3,2,1,0])

In [21]:
circ,qreg = circ_maker(n = n, qubits=qubits, f =False)
initial_layout = {}
for i in range(n):
    initial_layout[qreg[i]] = i

In [22]:
circ.draw()

┌───┐┌────────────────┐┌────────┐┌───┐┌───────┐┌──────────────┐     »
q17_0: ┤ X ├┤ Y_2^-1(1.4009) ├┤ Rz(-π) ├┤ X ├┤ Rz(π) ├┤ Y_2(-1.4009) ├──■──»
       ├───┤└────────────────┘└────────┘└─┬─┘└───────┘└──────────────┘┌─┴─┐»
q17_1: ┤ X ├──────────────────────────────■───────────────────────────┤ X ├»
       ├───┤┌───────────────┐ ┌────────┐┌───┐┌───────┐ ┌───────────┐  └───┘»
q17_2: ┤ X ├┤ Y_1^-1(-5π/4) ├─┤ Rz(-π) ├┤ X ├┤ Rz(π) ├─┤ Y_1(5π/4) ├────■──»
       ├───┤└───────────────┘ └────────┘└─┬─┘└───────┘ └───────────┘  ┌─┴─┐»
q17_3: ┤ X ├──────────────────────────────■───────────────────────────┤ X ├»
       └───┘                                                          └───┘»
q17_4: ────────────────────────────────────────────────────────────────────»
                                                                           »
«                                                                             »
«q17_0: ──────────────────────────────────────────────────────────────────────»
«            ┌─────────────────┐┌────────┐┌───┐┌───────┐┌───────────────┐     »
«q17_1: ──■──┤ Y_3^-1(0.61548) ├┤ Rz(-π) ├┤ X ├┤ Rz(π) ├┤ Y_3(-0.61548) ├──■──»
«       ┌─┴─┐└─────────────────┘└────────┘└─┬─┘└───────┘└───────────────┘┌─┴─┐»
«q17_2: ┤ X ├───────────────────────────────■────────────────────────────┤ X ├»
«       └───┘                                                            └───┘»
«q17_3: ──────────────────────────────────────────────────────────────────────»
«                                                                             »
«q17_4: ──────────────────────────────────────────────────────────────────────»
«                                                                             »
«            ┌─────────────────┐┌────────┐┌───┐┌───────┐┌─────────────┐     »
«q17_0: ──■──┤ Y_5^-1(-6.8068) ├┤ Rz(-π) ├┤ X ├┤ Rz(π) ├┤ Y_5(6.8068) ├──■──»
«       ┌─┴─┐└─────────────────┘└────────┘└─┬─┘└───────┘└─────────────┘┌─┴─┐»
«q17_1: ┤ X ├───────────────────────────────■──────────────────────────┤ X ├»
«       └───┘ ┌───────────────┐ ┌────────┐┌───┐┌───────┐ ┌───────────┐ └───┘»
«q17_2: ──■───┤ Y_4^-1(-3π/2) ├─┤ Rz(-π) ├┤ X ├┤ Rz(π) ├─┤ Y_4(3π/2) ├───■──»
«       ┌─┴─┐ └───────────────┘ └────────┘└─┬─┘└───────┘ └───────────┘ ┌─┴─┐»
«q17_3: ┤ X ├───────────────────────────────■──────────────────────────┤ X ├»
«       └───┘                                                          └───┘»
«q17_4: ────────────────────────────────────────────────────────────────────»
«                                                                           »
«                                                                             
«q17_0: ──────────────────────────────────────────────────────────────────────
«            ┌─────────────────┐┌────────┐┌───┐┌───────┐┌───────────────┐     
«q17_1: ──■──┤ Y_3^-1(0.61548) ├┤ Rz(-π) ├┤ X ├┤ Rz(π) ├┤ Y_3(-0.61548) ├──■──
«       ┌─┴─┐└─────────────────┘└────────┘└─┬─┘└───────┘└───────────────┘┌─┴─┐
«q17_2: ┤ X ├───────────────────────────────■────────────────────────────┤ X ├
«       └───┘                                                            └───┘
«q17_3: ──────────────────────────────────────────────────────────────────────
«                                                                             
«q17_4: ──────────────────────────────────────────────────────────────────────
«

In [23]:
a = qubits[0]
b = qubits[1]
c = qubits[2]
d = qubits[3]    

meas_calibs, state_labels = complete_meas_cal(qubit_list=qubits)
qst_circs = state_tomography_circuits(circ, [qreg[a],qreg[b], qreg[c], qreg[d]])

In [24]:
qst_circs = transpile(qst_circs, backend = backend, optimization_level = 0, initial_layout = initial_layout)
meas_calibs = transpile(meas_calibs,backend = backend)

In [25]:
job_manager = IBMQJobManager()

In [26]:
cal_job = job_manager.run(meas_calibs, backend = backend)
tomo_job = job_manager.run(qst_circs, backend = backend)

In [27]:
cal_job_r = cal_job.results()
tomo_job_r = tomo_job.results()

In [28]:
meas_fitter = CompleteMeasFitter(cal_job_r.combine_results(), state_labels)
tomo_fitter = StateTomographyFitter(tomo_job_r.combine_results(), qst_circs)

In [29]:
rho = tomo_fitter.fit()
F = state_fidelity(rho,target_state)
print("Fidelity without readout error mitigation: ",F)

Fidelity without readout error mitigation:  0.4007074475015424


In [30]:
correct_tomo_results = meas_fitter.filter.apply(tomo_job_r.combine_results(), method='least_squares')
tomo_mit = StateTomographyFitter(correct_tomo_results, qst_circs)
rho_mit = tomo_mit.fit(method='lstsq')
F_mit = state_fidelity(rho_mit, target_state)
print("Fidelity with readout error mitigation: ", F_mit)

Fidelity with readout error mitigation:  0.47677098695114384


In [56]:
saver(filename = filename, backend_name=backend_name, time = time, shots = shots, qubits = qubits, 
     F=F, rho_mes = rho, target_state = target_state, tomo_fitter_data=tomo_fitter.data, F_mit = F_mit,
     rho_mit = rho_mit, cal_matrix = meas_fitter.filter.cal_matrix)

## Santiago

In [10]:
backend = provider.get_backend('ibmq_santiago')
backend_name = 'Santiago'

time = "2021.04.26"
n=5
shots = 8192

filename = "Santiago4"
qubits = np.array([4,3,2,1])

In [11]:
circ,qreg = circ_maker(n = n, qubits=qubits, f =False)
initial_layout = {}
for i in range(n):
    initial_layout[qreg[i]] = i

In [12]:
circ.draw()

»
q19_0: ──────────────────────────────────────────────────────────────────»
       ┌───┐┌────────┐┌────────┐┌───┐┌───────┐┌─────┐                    »
q19_1: ┤ X ├┤ Y_2^-1 ├┤ RZ(-π) ├┤ X ├┤ RZ(π) ├┤ Y_2 ├──■─────────────────»
       ├───┤└────────┘└────────┘└─┬─┘└───────┘└─────┘┌─┴─┐     ┌────────┐»
q19_2: ┤ X ├──────────────────────■──────────────────┤ X ├──■──┤ Y_3^-1 ├»
       ├───┤┌────────┐┌────────┐┌───┐┌───────┐┌─────┐└───┘┌─┴─┐└────────┘»
q19_3: ┤ X ├┤ Y_1^-1 ├┤ RZ(-π) ├┤ X ├┤ RZ(π) ├┤ Y_1 ├──■──┤ X ├──────────»
       ├───┤└────────┘└────────┘└─┬─┘└───────┘└─────┘┌─┴─┐└───┘          »
q19_4: ┤ X ├──────────────────────■──────────────────┤ X ├───────────────»
       └───┘                                         └───┘               »
«                                                                         »
«q19_0: ──────────────────────────────────────────────────────────────────»
«                                                ┌────────┐┌────────┐┌───┐»
«q19_1: ──────────────────────────────────────■──┤ Y_5^-1 ├┤ RZ(-π) ├┤ X ├»
«       ┌────────┐┌───┐┌───────┐┌─────┐     ┌─┴─┐└────────┘└────────┘└─┬─┘»
«q19_2: ┤ RZ(-π) ├┤ X ├┤ RZ(π) ├┤ Y_3 ├──■──┤ X ├──────────────────────■──»
«       └────────┘└─┬─┘└───────┘└─────┘┌─┴─┐└───┘┌────────┐┌────────┐┌───┐»
«q19_3: ────────────■──────────────────┤ X ├──■──┤ Y_4^-1 ├┤ RZ(-π) ├┤ X ├»
«                                      └───┘┌─┴─┐└────────┘└────────┘└─┬─┘»
«q19_4: ────────────────────────────────────┤ X ├──────────────────────■──»
«                                           └───┘                         »
«                                                                          »
«q19_0: ───────────────────────────────────────────────────────────────────»
«       ┌───────┐┌─────┐                                                   »
«q19_1: ┤ RZ(π) ├┤ Y_5 ├──■────────────────────────────────────────────────»
«       └───────┘└─────┘┌─┴─┐     ┌────────┐┌────────┐┌───┐┌───────┐┌─────┐»
«q19_2: ────────────────┤ X ├──■──┤ Y_3^-1 ├┤ RZ(-π) ├┤ X ├┤ RZ(π) ├┤ Y_3 ├»
«       ┌───────┐┌─────┐└───┘┌─┴─┐└────────┘└────────┘└─┬─┘└───────┘└─────┘»
«q19_3: ┤ RZ(π) ├┤ Y_4 ├──■──┤ X ├──────────────────────■──────────────────»
«       └───────┘└─────┘┌─┴─┐└───┘                                         »
«q19_4: ────────────────┤ X ├──────────────────────────────────────────────»
«                       └───┘                                              »
«            
«q19_0: ─────
«            
«q19_1: ─────
«            
«q19_2: ──■──
«       ┌─┴─┐
«q19_3: ┤ X ├
«       └───┘
«q19_4: ─────
«

In [48]:
a = qubits[0]
b = qubits[1]
c = qubits[2]
d = qubits[3]    

meas_calibs, state_labels = complete_meas_cal(qubit_list=qubits)
qst_circs = state_tomography_circuits(circ, [qreg[a],qreg[b], qreg[c], qreg[d]])

In [49]:
qst_circs = transpile(qst_circs, backend = backend, optimization_level = 3, initial_layout = initial_layout)
meas_calibs = transpile(meas_calibs,backend = backend)

In [50]:
job_manager = IBMQJobManager()

In [51]:
cal_job = job_manager.run(meas_calibs, backend=backend)
tomo_job = job_manager.run(qst_circs, backend=backend)

/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/providers/ibmq/managed/managedjob.py:114: DeprecationWarning: run keyword argument qobj is deprecated and replaced with circuits.
  self.job = backend.run(
/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/providers/ibmq/managed/managedjob.py:114: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  self.job = backend.run(
/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/providers/ibmq/managed/managedjob.py:114: DeprecationWarning: run keyword argument qobj is deprecated and replaced with circuits.
  self.job = backend.run(
/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/providers/ibmq/managed/managedjob.py:114: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  self.job = backend.run(


In [52]:
cal_job_r = cal_job.results()
tomo_job_r = tomo_job.results()

/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/providers/ibmq/managed/managedjob.py:114: DeprecationWarning: run keyword argument qobj is deprecated and replaced with circuits.
  self.job = backend.run(
/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/providers/ibmq/managed/managedjob.py:114: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  self.job = backend.run(


In [53]:
meas_fitter = CompleteMeasFitter(cal_job_r.combine_results(), state_labels)
tomo_fitter = StateTomographyFitter(tomo_job_r.combine_results(), qst_circs)

In [54]:
rho = tomo_fitter.fit()
F = state_fidelity(rho,target_state)
print("Fidelity without readout error mitigation: ",F)

Fidelity without readout error mitigation:  0.6659364556915284


In [55]:
correct_tomo_results = meas_fitter.filter.apply(tomo_job_r.combine_results(), method='least_squares')
tomo_mit = StateTomographyFitter(correct_tomo_results, qst_circs)
rho_mit = tomo_mit.fit(method='lstsq')
F_mit = state_fidelity(rho_mit, target_state)
print("Fidelity with readout error mitigation: ", F_mit)

Fidelity with readout error mitigation:  0.7802556203451831


In [56]:
saver(filename = filename, backend_name=backend_name, time = time, shots = shots, qubits = qubits, 
     F=F, rho_mes = rho, target_state = target_state, tomo_fitter_data=tomo_fitter.data, F_mit = F_mit,
     rho_mit = rho_mit, cal_matrix = meas_fitter.filter.cal_matrix)

## Bogota

In [21]:
backend = provider.get_backend('ibmq_bogota')
backend_name = 'Bogota'

time = "2021.04.27"
n=5
shots = 8192

filename = "Bogota4"
qubits = np.array([4,3,2,1])

In [22]:
circ,qreg = circ_maker(n = n, qubits=qubits, f =False)
initial_layout = {}
for i in range(n):
    initial_layout[qreg[i]] = i

In [23]:
circ.draw()

»
q21_0: ──────────────────────────────────────────────────────────────────»
       ┌───┐┌────────┐┌────────┐┌───┐┌───────┐┌─────┐                    »
q21_1: ┤ X ├┤ Y_2^-1 ├┤ RZ(-π) ├┤ X ├┤ RZ(π) ├┤ Y_2 ├──■─────────────────»
       ├───┤└────────┘└────────┘└─┬─┘└───────┘└─────┘┌─┴─┐     ┌────────┐»
q21_2: ┤ X ├──────────────────────■──────────────────┤ X ├──■──┤ Y_3^-1 ├»
       ├───┤┌────────┐┌────────┐┌───┐┌───────┐┌─────┐└───┘┌─┴─┐└────────┘»
q21_3: ┤ X ├┤ Y_1^-1 ├┤ RZ(-π) ├┤ X ├┤ RZ(π) ├┤ Y_1 ├──■──┤ X ├──────────»
       ├───┤└────────┘└────────┘└─┬─┘└───────┘└─────┘┌─┴─┐└───┘          »
q21_4: ┤ X ├──────────────────────■──────────────────┤ X ├───────────────»
       └───┘                                         └───┘               »
«                                                                         »
«q21_0: ──────────────────────────────────────────────────────────────────»
«                                                ┌────────┐┌────────┐┌───┐»
«q21_1: ──────────────────────────────────────■──┤ Y_5^-1 ├┤ RZ(-π) ├┤ X ├»
«       ┌────────┐┌───┐┌───────┐┌─────┐     ┌─┴─┐└────────┘└────────┘└─┬─┘»
«q21_2: ┤ RZ(-π) ├┤ X ├┤ RZ(π) ├┤ Y_3 ├──■──┤ X ├──────────────────────■──»
«       └────────┘└─┬─┘└───────┘└─────┘┌─┴─┐└───┘┌────────┐┌────────┐┌───┐»
«q21_3: ────────────■──────────────────┤ X ├──■──┤ Y_4^-1 ├┤ RZ(-π) ├┤ X ├»
«                                      └───┘┌─┴─┐└────────┘└────────┘└─┬─┘»
«q21_4: ────────────────────────────────────┤ X ├──────────────────────■──»
«                                           └───┘                         »
«                                                                          »
«q21_0: ───────────────────────────────────────────────────────────────────»
«       ┌───────┐┌─────┐                                                   »
«q21_1: ┤ RZ(π) ├┤ Y_5 ├──■────────────────────────────────────────────────»
«       └───────┘└─────┘┌─┴─┐     ┌────────┐┌────────┐┌───┐┌───────┐┌─────┐»
«q21_2: ────────────────┤ X ├──■──┤ Y_3^-1 ├┤ RZ(-π) ├┤ X ├┤ RZ(π) ├┤ Y_3 ├»
«       ┌───────┐┌─────┐└───┘┌─┴─┐└────────┘└────────┘└─┬─┘└───────┘└─────┘»
«q21_3: ┤ RZ(π) ├┤ Y_4 ├──■──┤ X ├──────────────────────■──────────────────»
«       └───────┘└─────┘┌─┴─┐└───┘                                         »
«q21_4: ────────────────┤ X ├──────────────────────────────────────────────»
«                       └───┘                                              »
«            
«q21_0: ─────
«            
«q21_1: ─────
«            
«q21_2: ──■──
«       ┌─┴─┐
«q21_3: ┤ X ├
«       └───┘
«q21_4: ─────
«

In [24]:
a = qubits[0]
b = qubits[1]
c = qubits[2]
d = qubits[3]    

meas_calibs, state_labels = complete_meas_cal(qubit_list=qubits)
qst_circs = state_tomography_circuits(circ, [qreg[a],qreg[b], qreg[c], qreg[d]])

In [25]:
qst_circs = transpile(qst_circs, backend = backend, optimization_level = 3, initial_layout = initial_layout)
meas_calibs = transpile(meas_calibs,backend = backend)

In [26]:
job_manager = IBMQJobManager()

In [27]:
cal_job = job_manager.run(meas_calibs, backend=backend)
tomo_job = job_manager.run(qst_circs, backend=backend)

/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/providers/ibmq/managed/managedjob.py:114: DeprecationWarning: run keyword argument qobj is deprecated and replaced with circuits.
  self.job = backend.run(
/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/providers/ibmq/managed/managedjob.py:114: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  self.job = backend.run(
/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/providers/ibmq/managed/managedjob.py:114: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  self.job = backend.run(
/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/providers/ibmq/managed/managedjob.py:114: DeprecationWarning: run keyword argument qobj is deprecated and replaced with circuits.
  self.job = backend.run(
/home/

In [28]:
cal_job_r = cal_job.results()
tomo_job_r = tomo_job.results()

In [29]:
meas_fitter = CompleteMeasFitter(cal_job_r.combine_results(), state_labels)
tomo_fitter = StateTomographyFitter(tomo_job_r.combine_results(), qst_circs)

In [30]:
rho = tomo_fitter.fit()
F = state_fidelity(rho,target_state)
print("Fidelity without readout error mitigation: ",F)

Fidelity without readout error mitigation:  0.40029984614373926


In [31]:
correct_tomo_results = meas_fitter.filter.apply(tomo_job_r.combine_results(), method='least_squares')
tomo_mit = StateTomographyFitter(correct_tomo_results, qst_circs)
rho_mit = tomo_mit.fit(method='lstsq')
F_mit = state_fidelity(rho_mit, target_state)
print("Fidelity with readout error mitigation: ", F_mit)

Fidelity with readout error mitigation:  0.6268935868512479


In [32]:
saver(filename = filename, backend_name=backend_name, time = time, shots = shots, qubits = qubits, 
     F=F, rho_mes = rho, target_state = target_state, tomo_fitter_data=tomo_fitter.data, F_mit = F_mit,
     rho_mit = rho_mit, cal_matrix = meas_fitter.filter.cal_matrix)

## Rome

In [9]:
backend = provider.get_backend('ibmq_rome')
backend_name = 'Rome'

time = "2021.04.26"
n=5
shots = 8192

filename = "Rome3"
qubits = np.array([3,2,1,0])

In [10]:
circ,qreg = circ_maker(n = n, qubits=qubits, f =False)
initial_layout = {}
for i in range(n):
    initial_layout[qreg[i]] = i

In [11]:
circ.draw()

┌───┐┌────────┐┌────────┐┌───┐┌───────┐┌─────┐                    »
q19_0: ┤ X ├┤ Y_2^-1 ├┤ RZ(-π) ├┤ X ├┤ RZ(π) ├┤ Y_2 ├──■─────────────────»
       ├───┤└────────┘└────────┘└─┬─┘└───────┘└─────┘┌─┴─┐     ┌────────┐»
q19_1: ┤ X ├──────────────────────■──────────────────┤ X ├──■──┤ Y_3^-1 ├»
       ├───┤┌────────┐┌────────┐┌───┐┌───────┐┌─────┐└───┘┌─┴─┐└────────┘»
q19_2: ┤ X ├┤ Y_1^-1 ├┤ RZ(-π) ├┤ X ├┤ RZ(π) ├┤ Y_1 ├──■──┤ X ├──────────»
       ├───┤└────────┘└────────┘└─┬─┘└───────┘└─────┘┌─┴─┐└───┘          »
q19_3: ┤ X ├──────────────────────■──────────────────┤ X ├───────────────»
       └───┘                                         └───┘               »
q19_4: ──────────────────────────────────────────────────────────────────»
                                                                         »
«                                                ┌────────┐┌────────┐┌───┐»
«q19_0: ──────────────────────────────────────■──┤ Y_5^-1 ├┤ RZ(-π) ├┤ X ├»
«       ┌────────┐┌───┐┌───────┐┌─────┐     ┌─┴─┐└────────┘└────────┘└─┬─┘»
«q19_1: ┤ RZ(-π) ├┤ X ├┤ RZ(π) ├┤ Y_3 ├──■──┤ X ├──────────────────────■──»
«       └────────┘└─┬─┘└───────┘└─────┘┌─┴─┐└───┘┌────────┐┌────────┐┌───┐»
«q19_2: ────────────■──────────────────┤ X ├──■──┤ Y_4^-1 ├┤ RZ(-π) ├┤ X ├»
«                                      └───┘┌─┴─┐└────────┘└────────┘└─┬─┘»
«q19_3: ────────────────────────────────────┤ X ├──────────────────────■──»
«                                           └───┘                         »
«q19_4: ──────────────────────────────────────────────────────────────────»
«                                                                         »
«       ┌───────┐┌─────┐                                                   »
«q19_0: ┤ RZ(π) ├┤ Y_5 ├──■────────────────────────────────────────────────»
«       └───────┘└─────┘┌─┴─┐     ┌────────┐┌────────┐┌───┐┌───────┐┌─────┐»
«q19_1: ────────────────┤ X ├──■──┤ Y_3^-1 ├┤ RZ(-π) ├┤ X ├┤ RZ(π) ├┤ Y_3 ├»
«       ┌───────┐┌─────┐└───┘┌─┴─┐└────────┘└────────┘└─┬─┘└───────┘└─────┘»
«q19_2: ┤ RZ(π) ├┤ Y_4 ├──■──┤ X ├──────────────────────■──────────────────»
«       └───────┘└─────┘┌─┴─┐└───┘                                         »
«q19_3: ────────────────┤ X ├──────────────────────────────────────────────»
«                       └───┘                                              »
«q19_4: ───────────────────────────────────────────────────────────────────»
«                                                                          »
«            
«q19_0: ─────
«            
«q19_1: ──■──
«       ┌─┴─┐
«q19_2: ┤ X ├
«       └───┘
«q19_3: ─────
«            
«q19_4: ─────
«

In [12]:
a = qubits[0]
b = qubits[1]
c = qubits[2]
d = qubits[3]    

meas_calibs, state_labels = complete_meas_cal(qubit_list=qubits)
qst_circs = state_tomography_circuits(circ, [qreg[a],qreg[b], qreg[c], qreg[d]])

/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/ignis/verification/tomography/basis/circuits.py:468: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  prep += circuit
/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/circuit/quantumcircuit.py:876: DeprecationWarning: The QuantumCircuit.extend() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  return self.extend(rhs)
/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/ignis/verification/tomography/basis/circuits.py:478: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  circ = prep + meas
/home/zoltan/ana

In [13]:
qst_circs = transpile(qst_circs, backend = backend, optimization_level = 3, initial_layout = initial_layout)
meas_calibs = transpile(meas_calibs,backend = backend)

In [14]:
job_manager = IBMQJobManager()

In [15]:
cal_job = job_manager.run(meas_calibs, backend=backend)
tomo_job = job_manager.run(qst_circs, backend=backend)

/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/providers/ibmq/managed/managedjob.py:114: DeprecationWarning: run keyword argument qobj is deprecated and replaced with circuits.
  self.job = backend.run(
/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/providers/ibmq/managed/managedjob.py:114: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  self.job = backend.run(
/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/providers/ibmq/managed/managedjob.py:114: DeprecationWarning: run keyword argument qobj is deprecated and replaced with circuits.
  self.job = backend.run(
/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/providers/ibmq/managed/managedjob.py:114: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  self.job = backend.run(


In [16]:
cal_job_r = cal_job.results()
tomo_job_r = tomo_job.results()

In [17]:
meas_fitter = CompleteMeasFitter(cal_job_r.combine_results(), state_labels)
tomo_fitter = StateTomographyFitter(tomo_job_r.combine_results(), qst_circs)

In [18]:
rho = tomo_fitter.fit()
F = state_fidelity(rho,target_state)
print("Fidelity without readout error mitigation: ",F)

Fidelity without readout error mitigation:  0.14266133922987814


In [19]:
correct_tomo_results = meas_fitter.filter.apply(tomo_job_r.combine_results(), method='least_squares')
tomo_mit = StateTomographyFitter(correct_tomo_results, qst_circs)
rho_mit = tomo_mit.fit(method='lstsq')
F_mit = state_fidelity(rho_mit, target_state)
print("Fidelity with readout error mitigation: ", F_mit)

Fidelity with readout error mitigation:  0.15787340790350565


In [43]:
saver(filename = filename, backend_name=backend_name, time = time, shots = shots, qubits = qubits, 
     F=F, rho_mes = rho, target_state = target_state, tomo_fitter_data=tomo_fitter.data, F_mit = F_mit,
     rho_mit = rho_mit, cal_matrix = meas_fitter.filter.cal_matrix)

## Casablanca

In [36]:
backend = provider.get_backend('ibmq_casablanca')
backend_name = 'Casablanca'

time = "2021.04.28"
n=7
shots = 8192

filename = "Casablanca1"
qubits = np.array([0,1,3,5])

In [37]:
circ,qreg = circ_maker(n = n, qubits=qubits, f =False)
initial_layout = {}
for i in range(n):
    initial_layout[qreg[i]] = i

In [38]:
circ.draw()

┌───┐                                         ┌───┐               »
q24_0: ┤ X ├──────────────────────■──────────────────┤ X ├───────────────»
       ├───┤┌────────┐┌────────┐┌─┴─┐┌───────┐┌─────┐└─┬─┘┌───┐          »
q24_1: ┤ X ├┤ Y_1^-1 ├┤ RZ(-π) ├┤ X ├┤ RZ(π) ├┤ Y_1 ├──■──┤ X ├──────────»
       └───┘└────────┘└────────┘└───┘└───────┘└─────┘     └─┬─┘          »
q24_2: ─────────────────────────────────────────────────────┼────────────»
       ┌───┐                                         ┌───┐  │  ┌────────┐»
q24_3: ┤ X ├──────────────────────■──────────────────┤ X ├──■──┤ Y_3^-1 ├»
       └───┘                      │                  └─┬─┘     └────────┘»
q24_4: ───────────────────────────┼────────────────────┼─────────────────»
       ┌───┐┌────────┐┌────────┐┌─┴─┐┌───────┐┌─────┐  │                 »
q24_5: ┤ X ├┤ Y_2^-1 ├┤ RZ(-π) ├┤ X ├┤ RZ(π) ├┤ Y_2 ├──■─────────────────»
       └───┘└────────┘└────────┘└───┘└───────┘└─────┘                    »
q24_6: ──────────────────────────────────────────────────────────────────»
                                                                         »
«                                           ┌───┐                         »
«q24_0: ────────────────────────────────────┤ X ├──────────────────────■──»
«                                      ┌───┐└─┬─┘┌────────┐┌────────┐┌─┴─┐»
«q24_1: ────────────■──────────────────┤ X ├──■──┤ Y_4^-1 ├┤ RZ(-π) ├┤ X ├»
«                   │                  └─┬─┘     └────────┘└────────┘└───┘»
«q24_2: ────────────┼────────────────────┼────────────────────────────────»
«       ┌────────┐┌─┴─┐┌───────┐┌─────┐  │  ┌───┐                         »
«q24_3: ┤ RZ(-π) ├┤ X ├┤ RZ(π) ├┤ Y_3 ├──■──┤ X ├──────────────────────■──»
«       └────────┘└───┘└───────┘└─────┘     └─┬─┘                      │  »
«q24_4: ──────────────────────────────────────┼────────────────────────┼──»
«                                             │  ┌────────┐┌────────┐┌─┴─┐»
«q24_5: ──────────────────────────────────────■──┤ Y_5^-1 ├┤ RZ(-π) ├┤ X ├»
«                                                └────────┘└────────┘└───┘»
«q24_6: ──────────────────────────────────────────────────────────────────»
«                                                                         »
«                       ┌───┐                                              »
«q24_0: ────────────────┤ X ├──────────────────────────────────────────────»
«       ┌───────┐┌─────┐└─┬─┘┌───┐                                         »
«q24_1: ┤ RZ(π) ├┤ Y_4 ├──■──┤ X ├──────────────────────■──────────────────»
«       └───────┘└─────┘     └─┬─┘                      │                  »
«q24_2: ───────────────────────┼────────────────────────┼──────────────────»
«                       ┌───┐  │  ┌────────┐┌────────┐┌─┴─┐┌───────┐┌─────┐»
«q24_3: ────────────────┤ X ├──■──┤ Y_3^-1 ├┤ RZ(-π) ├┤ X ├┤ RZ(π) ├┤ Y_3 ├»
«                       └─┬─┘     └────────┘└────────┘└───┘└───────┘└─────┘»
«q24_4: ──────────────────┼────────────────────────────────────────────────»
«       ┌───────┐┌─────┐  │                                                »
«q24_5: ┤ RZ(π) ├┤ Y_5 ├──■────────────────────────────────────────────────»
«       └───────┘└─────┘                                                   »
«q24_6: ───────────────────────────────────────────────────────────────────»
«                                                                          »
«            
«q24_0: ─────
«       ┌───┐
«q24_1: ┤ X ├
«       └─┬─┘
«q24_2: ──┼──
«         │  
«q24_3: ──■──
«            
«q24_4: ─────
«            
«q24_5: ─────
«            
«q24_6: ─────
«

In [39]:
a = qubits[0]
b = qubits[1]
c = qubits[2]
d = qubits[3]    

meas_calibs, state_labels = complete_meas_cal(qubit_list=qubits)
qst_circs = state_tomography_circuits(circ, [qreg[a],qreg[b], qreg[c], qreg[d]])

In [40]:
qst_circs = transpile(qst_circs, backend = backend, optimization_level = 3, initial_layout = initial_layout)
meas_calibs = transpile(meas_calibs,backend = backend)

In [41]:
job_manager = IBMQJobManager()

In [42]:
cal_job = job_manager.run(meas_calibs, backend=backend)
tomo_job = job_manager.run(qst_circs, backend=backend)

/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/providers/ibmq/managed/managedjob.py:114: DeprecationWarning: run keyword argument qobj is deprecated and replaced with circuits.
  self.job = backend.run(
/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/providers/ibmq/managed/managedjob.py:114: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  self.job = backend.run(
/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/providers/ibmq/managed/managedjob.py:114: DeprecationWarning: run keyword argument qobj is deprecated and replaced with circuits.
  self.job = backend.run(
/home/zoltan/anaconda3/lib/python3.8/site-packages/qiskit/providers/ibmq/managed/managedjob.py:114: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  self.job = backend.run(


In [43]:
cal_job_r = cal_job.results()
tomo_job_r = tomo_job.results()

In [44]:
meas_fitter = CompleteMeasFitter(cal_job_r.combine_results(), state_labels)
tomo_fitter = StateTomographyFitter(tomo_job_r.combine_results(), qst_circs)

In [45]:
rho = tomo_fitter.fit()
F = state_fidelity(rho,target_state)
print("Fidelity without readout error mitigation: ",F)

Fidelity without readout error mitigation:  0.18117803412830827


In [46]:
correct_tomo_results = meas_fitter.filter.apply(tomo_job_r.combine_results(), method='least_squares')
tomo_mit = StateTomographyFitter(correct_tomo_results, qst_circs)
rho_mit = tomo_mit.fit(method='lstsq')
F_mit = state_fidelity(rho_mit, target_state)
print("Fidelity with readout error mitigation: ", F_mit)

Fidelity with readout error mitigation:  0.20705116529698017


In [47]:
saver(filename = filename, backend_name=backend_name, time = time, shots = shots, qubits = qubits, 
     F=F, rho_mes = rho, target_state = target_state, tomo_fitter_data=tomo_fitter.data, F_mit = F_mit,
     rho_mit = rho_mit, cal_matrix = meas_fitter.filter.cal_matrix)